In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when

# Start Spark session
spark = SparkSession.builder.getOrCreate()

# Load NEW CSV dataset
log_df = spark.read.option("header", True) \
    .option("inferSchema", True) \
    .csv("../Data/raw_logs/logdata.csv")

# Remove extra spaces in column names (safe step)
log_df = log_df.toDF(*[c.strip() for c in log_df.columns])

# Convert LogLevel to standardized log_level (INFO / ERROR)
log_df = log_df.withColumn(
    "log_level",
    when(col("LogLevel") == "INFO", "INFO").otherwise("ERROR")
)

# Total logs
print("Total logs:", log_df.count())

# Total errors
print("Total errors:", log_df.filter(col("log_level") == "ERROR").count())

# Error by type (based on LogLevel)
log_df.filter(col("log_level") == "ERROR") \
      .groupBy("LogLevel") \
      .count() \
      .orderBy("count", ascending=False) \
      .show()


Total logs: 100000
Total errors: 79858
+--------+-----+
|LogLevel|count|
+--------+-----+
| WARNING|26181|
|   DEBUG|25901|
|   ERROR|15851|
|   FATAL|11925|
+--------+-----+



In [2]:
log_df.filter(col("log_level") == "ERROR") \
      .groupBy("LogLevel") \
      .count() \
      .coalesce(1) \
      .write.mode("overwrite") \
      .csv("../data/reports/error_summary", header=True)


Py4JJavaError: An error occurred while calling o66.csv.
: java.lang.RuntimeException: java.io.FileNotFoundException: Could not locate Hadoop executable: C:\hadoop\bin\winutils.exe -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:286)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at or

In [9]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when

# 1️⃣ Set Hadoop home for Windows
os.environ["HADOOP_HOME"] = "C:\\hadoop"
os.environ["PATH"] += ";C:\\hadoop\\bin"

# 2️⃣ Start Spark session
spark = SparkSession.builder.getOrCreate()

# 3️⃣ Load CSV
log_df = spark.read.option("header", True).csv("../Data/raw_logs/Friday-WorkingHours-Afternoon-DDos.csv")

# 4️⃣ Strip spaces from column names
log_df = log_df.toDF(*[c.strip() for c in log_df.columns])

# 5️⃣ Create log_level column
log_df = log_df.withColumn("log_level", when(col("Label")=="BENIGN","INFO").otherwise("ERROR"))

# 6️⃣ Save error summary
log_df.filter(col("log_level")=="ERROR") \
      .groupBy("Label") \
      .count() \
      .coalesce(1) \
      .write.mode("overwrite") \
      .csv("../data/reports/error_summary", header=True)

print("Error summary saved successfully!")


Py4JJavaError: An error occurred while calling o150.csv.
: java.lang.RuntimeException: java.io.FileNotFoundException: Could not locate Hadoop executable: C:\hadoop\bin\winutils.exe -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:286)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at o

In [10]:
import os
os.environ["HADOOP_HOME"] = "C:\\hadoop"
os.environ["PATH"] += ";C:\\hadoop\\bin"


In [11]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when

# 1️⃣ Set Hadoop home for Windows
os.environ["HADOOP_HOME"] = "C:\\hadoop"
os.environ["PATH"] += ";C:\\hadoop\\bin"

# 2️⃣ Start Spark session
spark = SparkSession.builder.getOrCreate()

# 3️⃣ Load CSV
log_df = spark.read.option("header", True).csv("../Data/raw_logs/Friday-WorkingHours-Afternoon-DDos.csv")

# 4️⃣ Strip spaces from column names
log_df = log_df.toDF(*[c.strip() for c in log_df.columns])

# 5️⃣ Create log_level column
log_df = log_df.withColumn("log_level", when(col("Label")=="BENIGN","INFO").otherwise("ERROR"))

# 6️⃣ Save error summary
log_df.filter(col("log_level")=="ERROR") \
      .groupBy("Label") \
      .count() \
      .coalesce(1) \
      .write.mode("overwrite") \
      .csv("../data/reports/error_summary", header=True)

print("Error summary saved successfully!")


Py4JJavaError: An error occurred while calling o180.csv.
: java.lang.RuntimeException: java.io.FileNotFoundException: Could not locate Hadoop executable: C:\hadoop\bin\winutils.exe -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:286)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at o

In [1]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when

# 1️⃣ Set Hadoop home for Windows
os.environ["HADOOP_HOME"] = "C:\\hadoop"
os.environ["PATH"] += ";C:\\hadoop\\bin"

# 2️⃣ Start Spark session
spark = SparkSession.builder.getOrCreate()

# 3️⃣ Load CSV
log_df = spark.read.option("header", True).csv("../Data/raw_logs/Friday-WorkingHours-Afternoon-DDos.csv")

# 4️⃣ Strip spaces from column names
log_df = log_df.toDF(*[c.strip() for c in log_df.columns])

# 5️⃣ Create log_level column
log_df = log_df.withColumn("log_level", when(col("Label")=="BENIGN","INFO").otherwise("ERROR"))

# 6️⃣ Save error summary
log_df.filter(col("log_level")=="ERROR") \
      .groupBy("Label") \
      .count() \
      .coalesce(1) \
      .write.mode("overwrite") \
      .csv("../data/reports/error_summary", header=True)

print("Error summary saved successfully!")


Py4JJavaError: An error occurred while calling o49.csv.
: java.lang.RuntimeException: java.io.FileNotFoundException: Could not locate Hadoop executable: C:\hadoop\bin\winutils.exe -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:286)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at or

In [2]:
import os
os.environ["HADOOP_HOME"] = "C:\\hadoop"
os.environ["PATH"] += ";C:\\hadoop\\bin"

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()


In [3]:
import os
print(os.environ["HADOOP_HOME"])


C:\hadoop


In [4]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when

# 1️⃣ Set Hadoop home for Windows
os.environ["HADOOP_HOME"] = "C:\\hadoop"
os.environ["PATH"] += ";C:\\hadoop\\bin"

# 2️⃣ Start Spark session
spark = SparkSession.builder.getOrCreate()

# 3️⃣ Load CSV
log_df = spark.read.option("header", True).csv("../Data/raw_logs/Friday-WorkingHours-Afternoon-DDos.csv")

# 4️⃣ Strip spaces from column names
log_df = log_df.toDF(*[c.strip() for c in log_df.columns])

# 5️⃣ Create log_level column
log_df = log_df.withColumn("log_level", when(col("Label")=="BENIGN","INFO").otherwise("ERROR"))

# 6️⃣ Save error summary
log_df.filter(col("log_level")=="ERROR") \
      .groupBy("Label") \
      .count() \
      .coalesce(1) \
      .write.mode("overwrite") \
      .csv("../data/reports/error_summary", header=True)

print("Error summary saved successfully!")


Py4JJavaError: An error occurred while calling o81.csv.
: java.lang.RuntimeException: java.io.FileNotFoundException: Could not locate Hadoop executable: C:\hadoop\bin\winutils.exe -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:286)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at or

In [3]:
import pandas as pd
from pyspark.sql.functions import col

# Filter error logs and group by LogLevel (updated for new dataset schema)
error_summary = log_df.filter(col("log_level") == "ERROR") \
                      .groupBy("LogLevel") \
                      .count() \
                      .toPandas()

# Save CSV with Pandas
error_summary.to_csv("../data/reports/error_summary.csv", index=False)

print("Error summary saved successfully with Pandas!")


Error summary saved successfully with Pandas!


In [4]:
from pyspark.sql.functions import col

# Set threshold for alerts
ERROR_THRESHOLD = 1000  # adjust according to your dataset

# Count number of ERROR logs based on updated schema
error_count = log_df.filter(col("LogLevel") == "ERROR").count()

# Alert logic
if error_count > ERROR_THRESHOLD:
    print(f"ALERT: High number of ERROR logs detected! Count = {error_count}")
else:
    print(f"System stable. ERROR logs count = {error_count}")


ALERT: High number of ERROR logs detected! Count = 15851
